In [ ]:
from nltk.corpus import stopwords

import pandas as pd

import re

import textrank

import importlib
# importlib.reload(textrank)

from tqdm import tqdm

tqdm.pandas()

import multiprocessing as mp

import numpy as np

%matplotlib inline
import glob
import os

import spacy

nlp = spacy.load('en_core_web_sm')

import pickle

pd.options.display.max_colwidth=300

from collections import Counter

In [ ]:
df1 = pd.read_csv('/hn_more_20_2018_y.csv', index_col=0)
df1.head(1)

In [ ]:
docs = list(tqdm(nlp.pipe(df1['text'].values), total=df1.shape[0]))

In [ ]:
ents = list(map(lambda x: Counter([ent.label_ for ent in x.ents]), docs))

In [ ]:
with open('./data/preprocessed/spacy_ents_more_20_year_2019.pkl', 'wb') as f:
    pickle.dump(ents, f)

In [ ]:
df1.loc[:, 'ents'] = np.array(ents, dtype=object)

In [ ]:
df1.loc[:, 'ents_modal'] = df1['ents']\
    .progress_apply(lambda x: ' '.join(['{}:{}'.format(a, b) for a,b in x.items()]))

In [ ]:
matcher = Matcher(nlp.vocab)

pattern = [{'POS': 'PROPN', 'OP': '+'}, {'POS': 'PROPN', 'OP': '+'}]

matcher.add('prop_names', None, pattern)

In [ ]:
def pron_names_spacy(doc, matcher):
    return set([doc[start : end].text for match_id, start, end in matcher(doc)])

In [ ]:
%%time
df1['spacy_doc'].iloc[:1000].apply(lambda x: pron_names_spacy(x, matcher))

In [ ]:
df1['prop_names_spacy'] = df1['spacy_doc'].apply(lambda x: pron_names_spacy(x, matcher))